In [1]:
import simpy
import random
import numpy as np

In [2]:
seed = 978
random.seed(seed)

# Arrival mean                              --EXPONENTIAL
interarrival_mean = 6 

# Operator 1 mean and std for service time  --LOGNORMAL
m = 12 
s = 6
M = np.log(m**2/np.sqrt(m**2+s**2))
S = np.log((m**2+s**2)/m**2)

# Operator 2 service time range             --UNIFORM
service_range = [1,7]

# Voice recognition mean -                  --EXPONENTIAL
router_mean = 5

In [3]:
def service(env, opr):
    if opr==operator1:
        yield env.timeout(random.lognormvariate(M,S))
    elif opr==operator2:
        yield env.timeout(random.uniform(*service_range))

In [4]:
class Customer(object):
    def __init__(self, name, env):
        self.env = env
        self.name = name
        self.arrival_t = self.env.now
        self.action = env.process(self.call())
        
    
    def call(self):
        print('%s initiated a call at %g' % (self.name, self.env.now))
         
        # Voice recognition and routing
        yield env.timeout(random.expovariate(1/router_mean))
        if random.uniform(0,1) < .3:
            self.operator = operator1
        else:
            self.operator = operator2
            
        # Voice recognition failure
        if random.uniform(0,1) < .1:
            print('%s hangs up the call at %g' % (self.name, self.env.now))
            return 0 
            
        with self.operator.request() as req:
        
            yield req
            print('%s is assigned to the operator %d at %g' % (self.name,
                                                               (self.operator==operator2)+1,
                                                               self.env.now))
            
            yield self.env.process(service(env,self.operator))
            print('%s is done at %g' % (self.name, self.env.now))
        

In [5]:
def customer_generator(env):

    for i in range(10):
        yield env.timeout(random.expovariate(1/interarrival_mean))
        customer = Customer('Customer %s' %(i+1), env)  

In [6]:
env = simpy.Environment()
operator1 = simpy.Resource(env, capacity = 1)
operator2 = simpy.Resource(env, capacity = 1)
env.process(customer_generator(env))
env.run() 

Customer 1 initiated a call at 26.5136
Customer 1 is assigned to the operator 2 at 27.0016
Customer 2 initiated a call at 27.5241
Customer 1 is done at 30.6156
Customer 2 is assigned to the operator 2 at 36.3049
Customer 3 initiated a call at 37.4241
Customer 2 is done at 37.7309
Customer 3 is assigned to the operator 2 at 38.4629
Customer 3 is done at 41.6447
Customer 4 initiated a call at 49.3434
Customer 4 is assigned to the operator 2 at 55.8677
Customer 5 initiated a call at 59.6333
Customer 4 is done at 61.9611
Customer 5 is assigned to the operator 1 at 62.09
Customer 6 initiated a call at 71.3114
Customer 5 is done at 72.2459
Customer 6 is assigned to the operator 2 at 72.3254
Customer 6 is done at 73.4519
Customer 7 initiated a call at 87.0879
Customer 7 is assigned to the operator 2 at 87.6208
Customer 8 initiated a call at 91.8304
Customer 7 is done at 93.7976
Customer 8 is assigned to the operator 2 at 93.7976
Customer 8 is done at 99.2522
Customer 9 initiated a call at 102